In [4]:
import sys
import time
import csv
import math
import numpy as np
import pyspark
from pyspark import SparkContext, SparkConf
import json
from operator import add
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from node2vec import Node2Vec as n2v
import networkx as nx
from hashlib import md5
import pickle
import os
import pandas as pd
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 100)
pd.set_option("display.precision", 2)


from utils import create_category_md5_mapping, integrate_mapping_user_bus_cat_data, dataframe_to_rdd_dict, analyze_top_business_categories, analyze_top_categories

import json
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

def initialize_spark_context(APP_NAME="Train: XGBModel"):
    # Spark配置项列表
    SPARK_CONF = [
        ("spark.dynamicAllocation.enabled", "true"),  # 启用动态资源分配
        ("spark.dynamicAllocation.maxExecutors", "2"),  # 最大执行器数量
        ("spark.executor.memory", "4g"),  # 每个执行器的内存
        ("spark.executor.cores", "6"),  # 每个执行器的CPU核心数
        ("spark.executor.memoryOverhead", "3000"),  # 执行器内存开销
        ("spark.driver.memory", "4g"),  # 驱动程序的内存
        ("spark.driver.maxResultSize", "3"),  # 驱动程序的最大结果大小
        ("spark.python.worker.memory", "4g"),  # Python工作进程的内存
        ("spark.sql.shuffle.partitions", "20"),  # Shuffle操作的分区数
        ("spark.sql.sources.partitionOverWriteMode", "dynamic"),  # 分区覆写模式
        # ("spark.network.timeout", "10s"),  # 网络超时设置
        # ("spark.executor.heartbeatInterval", "10s"),  # 执行器心跳间隔
    ]

    # 创建Spark配置
    spark_conf = pyspark.SparkConf()
    spark_conf.setAppName(APP_NAME)
    spark_conf.setAll(SPARK_CONF)

    # 创建SparkContext
    sc = pyspark.SparkContext(conf=spark_conf)
    sc.setLogLevel("ERROR")  # 设置日志级别

    return sc

sc = initialize_spark_context()

24/05/04 00:33:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/04 00:33:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
import better_features
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data, extract_review_data
from datetime import datetime #add
import time
import pandas as pd

# def rdd_to_pandas(rdd):
#     return pd.DataFrame(rdd.collect(), columns=rdd.first().keys())

def prepare_test_data(test_df, cluster):
    if cluster == -1:
        test_cluster_data = test_df
    else:
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_test = test_cluster_data['stars']
    return X_test, y_test

start_time = time.time()

user_parsed_df = pd.read_csv('cache/user_df.csv') # parsed from users.json
business_parsed_df = pd.read_csv('cache/business_df.csv') # parsed from business.json
review_parsed_df = pd.read_csv('cache/review_df.csv') # parsed from business.json

print("DONE!")

DONE!


## SETUP

In [3]:
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data,extract_review_data
from utils import integrate_mapping_user_bus_cat_data
from KMeans_user_cluster import KMeans_process_user_clusters

data_folder_path, test_data_file, output_file = '../data/', '../data/yelp_true.csv', 'prediction.csv'
feature_processor = FeatureProcessor(sc, data_folder_path, user_parsed_df, business_parsed_df, review_parsed_df)
user_clusters = KMeans_process_user_clusters(feature_processor.map_reviews_with_categories(), business_parsed_df)

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [4]:
print("Duration: ", time.time() - start_time)

Duration:  157.96219611167908


In [5]:
######################################################

In [5]:
## TRAIN DATA FEATURE PROCESSING HERE
train_data_file = '../data/yelp_train.csv'
train_df = pd.read_csv(train_data_file)
train_df = feature_processor.process_all_features(sc, train_df, train_data_file)
train_df = train_df.merge(user_clusters, on='user_id', how='left')

print("DONE!")

DONE!


In [6]:
## TEST DATA FEATURE PROCESSING HERE
test_data_file = '../data/yelp_true.csv'
test_df = pd.read_csv(test_data_file)
test_df = feature_processor.process_all_features(sc, test_df, test_data_file)
test_df = test_df.merge(user_clusters, on='user_id', how='left')

print("DONE!")

DONE!


In [10]:
## TEST DATA FEATURE PROCESSING HERE
val_data_file = '../data/yelp_val.csv'
val_df = pd.read_csv(val_data_file)
val_df = feature_processor.process_all_features(sc, val_df, test_data_file)
val_df = val_df.merge(user_clusters, on='user_id', how='left')

print("DONE!")

DONE!


In [11]:
# train_df.to_csv('cache/train_df.csv', index=False)
# test_df.to_csv('cache/test_df.csv', index=False)
# val_df.to_csv('cache/val_df.csv', index=False)

# print("DONE!")

DONE!


## START

In [29]:
train_df[(train_df['score'] != 0) & (train_df['score'] < 0)]
test_df[(test_df['score'] != 0) & (test_df['score'] < 0)]

train_df[(train_df['score'] == 0)]
# train_df[(train_df['log_amp_affinity_score'] < 4.42)]
# train_df[(train_df['log_affinity_score'] > 1)]

,user_id,business_id,stars,score,average_stars,user_review_count,fans,useful,funny,cool,num_interactions,yelping_since,friends_count,elite_years,compliments,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_writer,compliment_photos,total_ufc_count_per_year,avg_ufc_count_per_review,bus_stars,bus_review_count,is_open,city_encoded,log_affinity_score,log_amp_affinity_score,Cluster
2,-qj9ouN0bzMXz1vfEslG-A,5j7BnXXvlS69uLVHrY9Upw,2.0,0.0,3.69,154,9,2660,644,1003,4307,12,55,0,160,12,6,0,0,0,15,39,37,14,0,36.15,7.22,3.5,45,1,416,0.44,4.41,0
8,QZ_Arlwoj0ghfBvg69rjOw,SUktrYdNQD8k2vvkM4OpfA,5.0,0.0,3.36,182,2,79,47,22,148,17,24,0,9,1,2,1,0,0,2,2,0,1,0,7.96,2.67,4.0,70,1,147,0.44,4.41,7
16,y0ZU1w6YY4W-KtMeRXSYLg,UpEVdq_euH-5mqCIgUAd2Q,4.0,0.0,3.58,231,10,1,5,2,8,16,35,0,76,8,2,2,0,1,11,15,8,15,6,7.70,3.14,3.5,133,1,984,0.44,4.41,0
19,tYk7mMGGFl3gLfmhST5L-A,cgoHLKJLsAK1ww0_ezNFnw,4.0,0.0,3.59,60,2,51,43,16,110,14,11,1,68,8,4,1,0,3,14,9,8,12,1,12.07,7.09,3.5,83,0,984,0.44,4.41,0
34,ehAvwDt4Fpsz9KPordu5lA,GPX3TnZ0-4pAxKPJUESbeA,3.0,0.0,3.86,170,3,27,1,28,56,14,299,0,2,0,0,0,0,0,1,0,0,1,0,6.41,0.80,4.0,155,1,333,0.44,4.41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455826,ubzSgqrV7dyLEQoHIocHXA,SipHumArj5ZIIpltsWwQpQ,4.0,0.0,3.09,88,1,35,29,19,83,13,3,0,18,2,0,1,0,0,2,4,2,5,0,4.98,2.04,4.0,75,0,984,0.44,4.41,0
455827,E6pqcEmf1LpL82-CIi60aw,_6n27rKvvDiPzWjcF24onA,3.0,0.0,3.86,301,4,8,1,0,9,10,24,3,41,2,2,0,0,0,4,6,8,9,2,12.03,2.17,4.5,123,1,893,0.44,4.41,3
455839,x16Fl_5h9aX4tn38kKSGHw,5LNZ67Yw9RD6nf4_UhXOjw,5.0,0.0,3.89,388,27,286,110,264,660,13,126,7,338,29,5,1,0,4,42,46,86,34,5,8.01,3.00,4.0,4097,1,416,0.44,4.41,0
455850,BrqGnby6aahIOc0N_1x0Bg,_UEQPDDiSgyYqjORWeLfJg,1.0,0.0,3.63,63,5,8,3,0,11,12,99,0,13,0,0,1,0,0,1,1,2,5,1,5.97,3.16,1.0,32,1,622,0.44,4.41,0


In [17]:
test_df[(test_df['score'] > 0)]
# test_df[(test_df['log_affinity_score'] > 1)]

,user_id,business_id,stars,score,average_stars,user_review_count,fans,useful,funny,cool,num_interactions,yelping_since,friends_count,elite_years,compliments,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_writer,compliment_photos,total_ufc_count_per_year,avg_ufc_count_per_review,bus_stars,bus_review_count,is_open,city_encoded,log_affinity_score,binary_affinity_score,Cluster
0,HzhIRYj_MHSyzer02AOBLA,DgE8JqFcYgp2SrjSKb3Gsg,4.0,3.42e-03,3.76,37,0,4,0,0,4,8,40,0,3,0,0,0,0,0,0,1,0,2,0,1.03,0.32,4.5,168,1,984,-37.77,1,0
1,DK01jsn-JcbiBamE1pB1mQ,8olG8J91NG_QU92WHG5QfA,5.0,1.91e-02,4.17,98,3,44,8,4,56,14,10,0,7,1,2,0,0,0,0,1,1,1,0,7.16,3.57,4.0,663,1,416,-37.50,1,0
2,456sZ9lPis-4EliHsWtQlw,TRM0H_3uKfbenY9TFtdhHg,1.0,1.86e-03,3.72,58,2,140,53,52,245,15,1,0,2,0,0,0,0,0,0,0,0,2,0,2.55,0.92,2.0,58,1,416,-37.80,1,0
4,hc71knjnXMI9wG_Oo9hpoA,qC9ivAdl47BcZdlbW7ffLg,3.0,3.75e-01,4.03,74,9,121,16,45,182,14,87,3,84,11,5,1,0,0,7,4,20,15,1,15.33,5.00,3.0,114,1,155,-29.34,1,0
5,zgV0ZroIF956gw4cul8MHA,AgMBY5tKMaFnFA2S6Vf22A,4.0,6.61e-01,3.90,608,151,2952,3816,2505,9273,14,523,7,2205,346,32,12,15,1,182,437,438,179,125,422.48,14.99,4.0,104,1,416,-19.05,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146990,keBv05MsMFBd0Hu98vXThQ,qEJjv2k8nDhOiYpmi_aVOw,5.0,3.19e-01,4.85,470,31,2818,343,756,3917,10,427,0,96,7,6,2,0,0,6,13,17,3,25,143.57,5.05,3.5,530,1,416,-30.90,1,0
146991,5ArMzPUBncPqDrB7PmZBCg,aYyj9OdH059CoEXadmldXA,5.0,2.38e-01,3.40,77,2,0,0,0,0,14,130,0,8,0,0,0,0,0,3,2,0,3,0,8.43,1.52,4.0,259,1,416,-32.98,1,0
146992,Fr6nMhzpaaicLsx_ZBczgg,CFtZH4Skp9z3o4ToSywI4w,4.0,1.09e+00,3.35,80,3,21,22,13,56,15,93,3,79,16,3,0,0,0,5,8,16,15,0,26.03,10.87,3.5,422,1,725,6.86,1,0
146993,yQiFb3kzronkldMDomOAWg,pk3C3zEvBWl8Pfby1HjvUw,2.0,1.78e-01,3.61,157,2,12,1,1,14,10,6,0,3,0,1,0,0,0,1,1,0,0,0,3.46,0.64,3.5,81,1,333,-34.34,1,0


## START HERE

In [6]:
train_df = pd.read_csv('cache/train_df.csv', index_col=None)
test_df = pd.read_csv('cache/test_df.csv', index_col=None)
val_df = pd.read_csv('cache/val_df.csv', index_col=None)

In [4]:
train_df.columns

Index(['user_id', 'business_id', 'stars', 'score', 'average_stars', 'user_review_count', 'fans',
       'useful', 'funny', 'cool', 'num_interactions', 'yelping_since', 'friends_count',
       'elite_years', 'compliments', 'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_writer', 'compliment_photos', 'total_ufc_count_per_year',
       'avg_ufc_count_per_review', 'bus_stars', 'bus_review_count', 'is_open', 'city_encoded',
       'log_affinity_score', 'binary_affinity_score', 'Cluster'],
      dtype='object')

## ATTEMPT: MODEL ENSEMBLING

- Large Sample Clusters: Cluster 0, 2, 4, and 3 have relatively higher sample counts ranging from about 8,189 to 559,331.
- Moderate Sample Clusters: Clusters 7, 6, and 8 have moderate numbers ranging from about 1,961 to 2,153.
- Small Sample Clusters: Clusters 5 and 1 are significantly smaller, with 1,332 and 82 samples, respectively.

Sample counts per Cluster:
 Cluster
0    422982
2     12745
4      8183
3      6189
7      1632
6      1570
8      1487
5      1004
1        62
  
**early_stopping_rounds=10**

{0: 0.9826888034981346, 1: 1.014988119592079, 2: 0.8756094886713228, 3: 0.9577187369705044, 4: 0.9463867369241324, 
 5: 1.0896717324335012, 6: 0.9023332514439544, 7: 1.0593137926993073, 8: 1.0335643721328076}

In [36]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

models = {}
rmse_scores = {}

# default params, the best found on train dataset
params = {'objective': 'reg:squarederror', 'max_depth': 7, 'n_estimators': 400, 'min_child_weight': 4, 'colsample_bytree': 0.6827530025351433, 
       'learning_rate': 0.03958467377002343, 'gamma': 0.699572436949439, 'subsample': 0.8634832107993408}

In [37]:
params

{'objective': 'reg:squarederror',
 'max_depth': 7,
 'n_estimators': 400,
 'min_child_weight': 4,
 'colsample_bytree': 0.6827530025351433,
 'learning_rate': 0.03958467377002343,
 'gamma': 0.699572436949439,
 'subsample': 0.8634832107993408}

In [45]:
'''test on train and test data'''
from sklearn.model_selection import train_test_split

# default params, the best found on train dataset
params = {'objective': 'reg:squarederror', 'max_depth': 7, 'n_estimators': 400, 'min_child_weight': 4, 'colsample_bytree': 0.6827530025351433, 
       'learning_rate': 0.03958467377002343, 'gamma': 0.699572436949439, 'subsample': 0.8634832107993408}

for cluster in range(9):
    train_cluster_data = train_df[train_df['Cluster'] == cluster]
    X = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id']) # , 'log_affinity_score'
    y = train_cluster_data['stars']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42)
    
    # 使用XGBRegressor
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)

    def prepare_test_data(test_df, cluster):
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
        y_test = test_cluster_data['stars']
        return X_test, y_test

    X_test, y_test = prepare_test_data(test_df, cluster)
    
    models[cluster] = model
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    rmse_scores[cluster] = rmse

# 输出每个Cluster的RMSE
print("RMSE Scores by Cluster:", rmse_scores)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_pa

RMSE Scores by Cluster: {0: 0.9826888034981346, 1: 1.014988119592079, 2: 0.8756094886713228, 3: 0.9577187369705044, 4: 0.9463867369241324, 5: 1.0896717324335012, 6: 0.9023332514439544, 7: 1.0593137926993073, 8: 1.0335643721328076}


/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [ ]:
{0: 0.9826888034981346, 1: 1.014988119592079, 2: 0.8756094886713228, 3: 0.9577187369705044, 4: 0.9463867369241324, 
 5: 1.0896717324335012, 6: 0.9023332514439544, 7: 1.0593137926993073, 8: 1.0335643721328076}

'''compare'''
***RMSE Scores by Cluster: {0: 0.9824686714564024, 1: 1.014988119592079, 2: 0.8767446977193243, 3: 0.9585491891701624, 4: 0.9452038088353173, 
                         5: 1.0873296412466678, 6: 0.907313500532493, 7: 1.068930368341315, 8: 1.029497789897289} # original w/ rank=120, default1=0, default2=1, +log_affinity_score, test_size=0.05

RMSE Scores by Cluster: {0: 0.9825338155446997, 1: 0.9901830991049633, 2: 0.8783358989800031, 3: 0.9578945444082837, 4: 0.9499816858920407, 
                         5: 1.0853201164187696, 6: 0.909600206073975, 7: 1.0549750413779833, 8: 1.0459203589186783} # original w/ rank=120, default=1, +log_affinity_score

***RMSE Scores by Cluster: {0: 0.982434545359731, 1: 1.0450832466807036, 2: 0.875241570724334, 3: 0.9612956183026671, 4: 0.948474842542706, 
                         5: 1.0710061004503517, 6: 0.8998862025792799, 7: 1.0382836776951188, 8: 1.0341570676457565} # original w/ rank=120, default=1, \log_affinity_score, test_size=0.1

RMSE Scores by Cluster: {0: 0.9828531835050012, 1: 1.0450832466807036, 2: 0.8769163030877263, 3: 0.9582877665467199, 4: 0.9492225823191563, 
                         5: 1.070343649997305, 6: 0.9013046327620369, 7: 1.0499606367320111, 8: 1.0384354316973448} # default=0, \log_affinity_score

RMSE Scores by Cluster: {0: 0.9830479933504006, 1: 1.0450832466807036, 2: 0.8791304453718787, 3: 0.9629053092798874, 4: 0.948741200048593, 
                         5: 1.0707645862520345, 6: 0.9096426863912603, 7: 1.0576515429369537, 8: 1.0400421276418648} # default=1, \log_affinity_score

RMSE Scores by Cluster: {0: 0.9834585097590449, 1: 1.0561494279772126, 2: 0.8765824445741225, 3: 0.9659717158520954, 4: 0.9467519455256139, 
                         5: 1.0610768207383312, 6: 0.9046608167556579, 7: 1.039620970687066, 8: 1.0359433156935904}


# original
RMSE Scores by Cluster: {0: 0.9828658502650812, 2: 0.8761593887238531, 4: 0.9443702348479344, 3: 0.9689126759366788, 
1: 0.9486380392367105, 5: 1.0693467025903196, # with log_affinity_score * 50
6: 0.9071186256407098, 7: 1.042741917361027, 8: 1.039941487282683} # original 

In [16]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# 初始化一个空列表，用于存储每个群集的预测结果DataFrame
all_clusters_predictions = []

for cluster in range(9):
    train_cluster_data = train_df[train_df['Cluster'] == cluster]
    X = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y = train_cluster_data['stars']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42)

    test_cluster_data = test_df[test_df['Cluster'] == cluster]
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_test = test_cluster_data['stars']
    
    # 使用XGBRegressor
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
    
    models[cluster] = model
    predictions = model.predict(X_test)
    
    # 创建预测结果的DataFrame
    temp_df = pd.DataFrame({
        'user_id': test_cluster_data['user_id'],
        'business_id': test_cluster_data['business_id'],
        'stars': y_test,
        'prediction': predictions
    })
    all_clusters_predictions.append(temp_df)

# 将所有群集的预测结果合并成一个DataFrame
all_predictions = pd.concat(all_clusters_predictions, ignore_index=True)
all_predictions['prediction'] = all_predictions['prediction'].clip(lower=1, upper=5)

# 计算整个DataFrame的RMSE
overall_rmse = np.sqrt(mean_squared_error(all_predictions['stars'], all_predictions['prediction']))
print("Overall RMSE:", overall_rmse)

# 输出合并后的DataFrame
print(all_predictions.head())

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_pa

Overall RMSE: 0.9792850242342954
                  user_id             business_id  stars  prediction
0  HzhIRYj_MHSyzer02AOBLA  DgE8JqFcYgp2SrjSKb3Gsg    4.0        4.36
1  DK01jsn-JcbiBamE1pB1mQ  8olG8J91NG_QU92WHG5QfA    5.0        4.18
2  456sZ9lPis-4EliHsWtQlw  TRM0H_3uKfbenY9TFtdhHg    1.0        2.38
3  4bgpE8kMw8cQCaMin_dcww  QEgvfB8OUq5_X-UURbw2DA    4.0        4.53
4  hc71knjnXMI9wG_Oo9hpoA  qC9ivAdl47BcZdlbW7ffLg    3.0        3.38


/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [20]:
import pandas as pd

# 假设final_df是您的整个数据集
# 统计每个Cluster的样本数量
sample_counts = train_df['Cluster'].value_counts()
print("Sample counts per Cluster:\n", sample_counts)

# 分析特征分布
feature_analysis = {}
for cluster in range(9):
    cluster_data = train_df[train_df['Cluster'] == cluster]
    feature_description = cluster_data.describe()
    feature_analysis[cluster] = feature_description

# 打印出每个Cluster的描述统计，查看是否有异常值或分布不平衡
for cluster, description in feature_analysis.items():
    print(f"Cluster {cluster} - Feature Statistics:\n{description}\n")


Sample counts per Cluster:
 Cluster
0    136349
2      4136
4      2653
3      2000
7       521
6       515
8       474
5       328
1        20
Name: count, dtype: int64
Cluster 0 - Feature Statistics:
           stars     score  average_stars  user_review_count       fans    useful      funny  \
count  136349.00  1.36e+05      136349.00          136349.00  136349.00  136349.0  136349.00   
mean        3.74  3.62e-01           3.75             362.84      39.53     958.0     561.44   
std         1.13  3.93e-01           0.39             650.91     103.65    5445.7    3922.46   
min         1.00  0.00e+00           1.42              31.00       0.00       0.0       0.00   
25%         3.00  6.57e-03           3.53              87.00       3.00      12.0       2.00   
50%         4.00  2.11e-01           3.77             165.00      10.00      55.0      13.00   
75%         5.00  6.54e-01           4.00             374.00      31.00     262.0      91.00   
max         5.00  7.09e+00    

In [39]:
sample_counts = train_df['Cluster'].value_counts()
print("Sample counts per Cluster:\n", sample_counts)

Sample counts per Cluster:
 Cluster
0    422982
2     12745
4      8183
3      6189
7      1632
6      1570
8      1487
5      1004
1        62
Name: count, dtype: int64


## APPROACH 1: FOR EACH CLUSTER, BUILD A MODEL to get params and best-suited model

In [ ]:
# search_spaces = {
#     'max_depth': Integer(7, 9),
#     'learning_rate': Real(0.029, 0.035, 'uniform'),
#     'n_estimators': Integer(470, 490),
#     'subsample': Real(0.87, 0.93),
#     'colsample_bytree': Real(0.58, 0.61),
#     'gamma': Real(0.34, 0.40),
#     'max_leaves': Integer(118, 126)
# }
# 最佳参数: OrderedDict([('colsample_bytree', 0.6000253271077364), 
#                    ('gamma', 0.3476073435753331), 
#                    ('learning_rate', 0.03186356519406673), 
#                    ('max_depth', 8), 
#                    ('n_estimators', 480), 
#                    ('subsample', 0.8831766281686719)]) 
# 最终测试集上的RMSE: 0.9472496841333367

### Large Sample Clusters (0, 2, 4, 3)

In [21]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
# params = {'objective': 'reg:squarederror', 'max_depth': 7, 'n_estimators': 400, 'min_child_weight': 4, 'colsample_bytree': 0.6827530025351433, 
#        'learning_rate': 0.03958467377002343, 'gamma': 0.699572436949439, 'subsample': 0.8634832107993408}

# 定义搜索空间
space = {
    'max_depth': hp.choice('max_depth', range(6, 12)),
    'n_estimators': hp.choice('n_estimators', range(390, 410, 10)),
    'min_child_weight': hp.choice('min_child_weight', range(3, 7)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.60, 0.70),
    'learning_rate': hp.uniform('learning_rate', 0.02, 0.04),
    'gamma': hp.uniform('gamma', 0.60, 0.70),
    'subsample': hp.uniform('subsample', 0.84, 0.88)
}

def objective(params):
    model = XGBRegressor(
        objective='reg:squarederror',
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        max_leaves=int(params['max_leaves'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return {'loss': rmse, 'status': STATUS_OK}

best_models = {}
rmse_scores = {}

for cluster in [0, 2, 4, 3]:
    train_cluster_data = final_df[final_df['Cluster'] == cluster]
    X_train = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_train = train_cluster_data['stars']
    
    val_cluster_data = val_df[val_df['Cluster'] == cluster]
    X_val = val_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_val = val_cluster_data['stars']

    trials = Trials()
    best = fmin(
        fn=objective,
        space=search_spaces,
        algo=tpe.suggest,
        max_evals=50,  # 可以根据需要调整迭代次数
        trials=trials
    )
    
    best_model = XGBRegressor(
        objective='reg:squarederror',
        max_depth=range(7, 10)[best['max_depth']],
        learning_rate=best['learning_rate'],
        n_estimators=range(420, 481)[best['n_estimators']],
        subsample=best['subsample'],
        colsample_bytree=best['colsample_bytree'],
        gamma=best['gamma'],
        max_leaves=range(118, 127)[best['max_leaves']]
    )
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_val)
    final_rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    best_models[cluster] = best_model
    rmse_scores[cluster] = final_rmse

    print(f"Cluster {cluster} - 最佳参数: {best}")
    print(f"Cluster {cluster} - 最终测试集上的RMSE: {final_rmse}")


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:06<00:00,  4.93s/trial, best loss: 0.9822613434809238]
Cluster 0 - 最佳参数: {'colsample_bytree': 0.5693794662243868, 'gamma': 0.3540225200080566, 'learning_rate': 0.03264647132113966, 'max_depth': 1, 'max_leaves': 1, 'n_estimators': 15, 'subsample': 0.8806235635005766}
Cluster 0 - 最终测试集上的RMSE: 0.9822613434809238
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:46<00:00,  1.08trial/s, best loss: 0.8782371474956998]
Cluster 2 - 最佳参数: {'colsample_bytree': 0.5637401106046143, 'gamma': 0.3516260477605038, 'learning_rate': 0.02015222693767021, 'max_depth': 0, 'max_leaves': 7, 'n_estimators': 8, 'subsample': 0.9177283905231816}
Cluster 2 - 最终测试集上的RMSE: 0.8782371474956998
100%|██████████████████████████████████████████████

In [ ]:
# RMSE Scores by Cluster: {0: 0.9822314440754664, 1: 0.9332088716462881, 2: 0.8751002637935721, 3: 0.9574858709357046, 4: 0.9404585183198316, 
#                          5: 1.0846576627248898, 6: 0.8896547222491545, 7: 1.0397995259518373, 8: 1.0388345770749272}

# # default params, the best found on train dataset
# params = {'objective': 'reg:squarederror', 'max_depth': 7, 'n_estimators': 400, 
#           'min_child_weight': 4, 'colsample_bytree': 0.6827530025351433, 
#        'learning_rate': 0.03958467377002343, 'gamma': 0.699572436949439, 'subsample': 0.8634832107993408}

In [57]:
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


# 准备测试数据
X_train, y_train = prepare_test_data(train_df, 3) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

X_test, y_test = prepare_test_data(test_df, 3)

def xgb_eval(max_depth, learning_rate, n_estimators, subsample): # , gamma, min_child_weight, colsample_bytree
    model = XGBRegressor(
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample=subsample,
        # gamma=gamma,
        # min_child_weight=min_child_weight,
        # colsample_bytree=colsample_bytree,
        objective='reg:squarederror',
        verbosity=0,  # 使用正确的参数控制输出
        early_stopping_rounds=10
    )
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # 使用测试数据评估模型
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    # 返回负RMSE，因为贝叶斯优化默认是最大化目标函数
    return -rmse

# 参数边界
pbounds = {
    'max_depth': (5, 6),
    'learning_rate': (0.08, 0.1),
    'n_estimators': (250, 320),
    'subsample': (0.5, 1.0),
    # 'gamma': (0.65, 0.75),
    # 'min_child_weight': (3, 5),
    # 'colsample_bytree': (0.67, 0.7),
}

optimizer = BayesianOptimization(f=xgb_eval, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=100, n_iter=50)

print("Best parameters:", optimizer.max['params'])

|   iter    |  target   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------
| 1         | -0.9508   | 0.08749   | 5.951     | 301.2     | 0.7993    |
| 2         | -0.9503   | 0.08312   | 5.156     | 254.1     | 0.9331    |
| 3         | -0.9519   | 0.09202   | 5.708     | 251.4     | 0.985     |
| 4         | -0.9518   | 0.09665   | 5.212     | 262.7     | 0.5917    |
| 5         | -0.9495   | 0.08608   | 5.525     | 280.2     | 0.6456    |
| 6         | -0.9501   | 0.09224   | 5.139     | 270.5     | 0.6832    |
| 7         | -0.9542   | 0.08912   | 5.785     | 264.0     | 0.7571    |
| 8         | -0.9557   | 0.09185   | 5.046     | 292.5     | 0.5853    |
| 9         | -0.9518   | 0.0813    | 5.949     | 317.6     | 0.9042    |
| 10        | -0.9539   | 0.08609   | 5.098     | 297.9     | 0.7201    |
| 11        | -0.9493   | 0.08244   | 5.495     | 252.4     | 0.9547    |
| 12        | -0.9509   | 0.08518   | 

In [ ]:
3
Best parameters: {'learning_rate': 0.08288064461501718, 'max_depth': 4.704925810006415, 
                  'n_estimators': 304.68373839140463, 'subsample': 0.6838579015297168}

-0.9463

# Best parameters: {'learning_rate': 0.07173423081368346, 'max_depth': 3.9789055205551263, 
#                   'n_estimators': 318.56504541106005, 'subsample': 0.8484110543023001}
# -0.9478

4
Best parameters: {'learning_rate': 0.18299396047960448, 'max_depth': 3.717897930418724, 
                  'n_estimators': 249.53928852131315, 'subsample': 0.9081495526065285}
-0.9327

2
Best parameters: {'learning_rate': 0.12350751460907078, 'max_depth': 3.097672114006384, 
                  'n_estimators': 288.4233026512157, 'subsample': 0.8880304987479203}
-0.8718

# Best parameters: {'learning_rate': 0.12350751460907078, 'max_depth': 3.097672114006384, 
#                   'n_estimators': 302.63495397682357, 'subsample': 0.8880304987479203}
# -0.8718

# Best parameters: {'learning_rate': 0.09913727843230384, 'max_depth': 3.798413063437975, 
#                   'n_estimators': 314.5428334929617, 'subsample': 0.8343921895955786}
# -0.8718

# Best parameters: {'learning_rate': 0.052015138549577726, 'max_depth': 3.040366245098638, 
#                   'n_estimators': 282.65168237360854, 'subsample': 0.36408083754707765}
# -0.872


0

Best parameters: {'learning_rate': 0.04246778091879101, 'max_depth': 6.332694494015188, 
                  'n_estimators': 491.29196986785007, 'subsample': 0.8689265095175513}

-0.9824

# Best parameters: {'learning_rate': 0.04246778091879101, 'max_depth': 6.332694494015188, 
#                   'n_estimators': 444.1946465785667, 'subsample': 0.8689265095175513}


In [25]:
from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split


X_train, y_train = prepare_test_data(train_df, 2) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

X_test, y_test = prepare_test_data(test_df, 2)
# 定义目标函数
def catboost_eval(depth, l2_leaf_reg, learning_rate, n_estimators):
    params = {
        'depth': int(depth),
        'l2_leaf_reg': l2_leaf_reg,
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'loss_function': 'RMSE',
        'verbose': False
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return -rmse

# 参数边界
pbounds = {
    'depth': (6, 7),  # 紧缩depth的范围
    'l2_leaf_reg': (10, 20),  # 在更小的l2_leaf_reg范围内搜索
    'learning_rate': (0.3, 0.6),  # 根据前面的结果，集中在较高的学习率
    'n_estimators': (120, 180)  # 根据有效范围调整
}

# 实例化贝叶斯优化对象
optimizer = BayesianOptimization(f=catboost_eval, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=200, n_iter=50)

print("Best parameters found:", optimizer.max['params'])

|   iter    |  target   |   depth   | l2_lea... | learni... | n_esti... |
-------------------------------------------------------------------------
| 1         | -0.8753   | 6.375     | 19.51     | 0.5196    | 155.9     |
| 2         | -0.8746   | 6.156     | 11.56     | 0.3174    | 172.0     |
| 3         | -0.8756   | 6.601     | 17.08     | 0.3062    | 178.2     |
| 4         | -0.8759   | 6.832     | 12.12     | 0.3545    | 131.0     |
| 5         | -0.8773   | 6.304     | 15.25     | 0.4296    | 137.5     |
| 6         | -0.8762   | 6.612     | 11.39     | 0.3876    | 142.0     |
| 7         | -0.8747   | 6.456     | 17.85     | 0.3599    | 150.9     |
| 8         | -0.8745   | 6.592     | 10.46     | 0.4823    | 130.2     |
| 9         | -0.8781   | 6.065     | 19.49     | 0.5897    | 168.5     |
| 10        | -0.8749   | 6.305     | 10.98     | 0.5053    | 146.4     |
| 11        | -0.8734   | 6.122     | 14.95     | 0.3103    | 174.6     |
| 12        | -0.8769   | 6.259     | 

In [ ]:
4 (searchspace from 3)
Best parameters found: {'depth': 4.011353644767419, 'l2_leaf_reg': 0.21811963851964758, 
                        'learning_rate': 0.19112606551363676, 'n_estimators': 52.9704479067018}
-0.9314

2
Best parameters found: {'depth': 6.467060867155791, 'l2_leaf_reg': 17.065903814470666, 
                        'learning_rate': 0.3860353293082986, 'n_estimators': 145.27827823278525}
-0.8712

In [168]:
xgb_params = {
    0: {'learning_rate': 0.04246778091879101, 'max_depth': 6, 'n_estimators': 491, 'subsample': 0.8689},
    2: {'learning_rate': 0.12350751460907078, 'max_depth': 3, 'n_estimators': 288, 'subsample': 0.8880},
    3: {'learning_rate': 0.08288064461501718, 'max_depth': 5, 'n_estimators': 305, 'subsample': 0.6839},
    4: {'learning_rate': 0.18299396047960448, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9081}
}

catboost_params = {
    0: {'depth': 12, 'l2_leaf_reg': 18.15, 'learning_rate': 0.06229, 'n_estimators': 1486},
    2: {'depth': 6, 'l2_leaf_reg': 28.02, 'learning_rate': 0.1090, 'n_estimators': 176},
    3: {'depth': 5, 'l2_leaf_reg': 0.2358, 'learning_rate': 0.1968, 'n_estimators': 62},
    4: {'depth': 4, 'l2_leaf_reg': 0.3376, 'learning_rate': 0.1914, 'n_estimators': 46}
}

from sklearn.metrics import mean_squared_error
import numpy as np

def find_best_weight(cluster, X_train, y_train, X_test, y_test):
    # 初始化模型
    cb_model = CatBoostRegressor(**catboost_params[cluster], verbose=False)
    xgb_model = XGBRegressor(**xgb_params[cluster], objective='reg:squarederror', verbosity=0)

    # 训练模型
    cb_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)

    # 预测
    cb_preds = cb_model.predict(X_test).clip(1, 5)
    xgb_preds = xgb_model.predict(X_test).clip(1, 5)

    best_rmse = float('inf')
    best_ratio = 0
    
    for ratio in np.linspace(0, 1, 21):  # 测试21个不同的比例
        final_preds = ratio * cb_preds + (1 - ratio) * xgb_preds
        rmse = np.sqrt(mean_squared_error(y_test, final_preds))
        if rmse < best_rmse:
            best_rmse = rmse
            best_ratio = ratio

    return best_ratio, best_rmse

# Example usage
cluster = 0
X_train, y_train = prepare_test_data(train_df, cluster) 
X_test, y_test = prepare_test_data(val_df, cluster)
best_ratio, best_rmse = find_best_weight(cluster, X_train, y_train, X_test, y_test)
print(f"Best Cat/Xgb ratio for cluster {cluster}: {best_ratio} with RMSE: {best_rmse}")

Best Cat/Xgb ratio for cluster 0: 0.7000000000000001 with RMSE: 0.9864837540160966


In [ ]:
Best Cat/Xgb ratio for cluster 0: 0.30 with RMSE: 0.9815215776560651
Best Cat/Xgb ratio for cluster 2: 0.65 with RMSE: 0.8709046585277763


#-------------------use catboost only
Best Cat/Xgb ratio for cluster 4: 0.8 with RMSE: 0.9325884893451591
Best Cat/Xgb ratio for cluster 3: 0.85 with RMSE: 0.9466939722254089

In [112]:
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split
# import numpy as np

# def prepare_data(df, cluster):
#     cluster_data = df[df['Cluster'] == cluster] if cluster != -1 else df
#     X = cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
#     y = cluster_data['stars']
#     return X, y

# def train_and_feature_importance(cluster, train_df, test_df, best_params):
#     # 准备数据
#     X_train, y_train = prepare_data(train_df, cluster)
#     X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)
#     X_test, y_test = prepare_data(test_df, cluster)
    
#     # 初始化模型
#     params = best_params[cluster]
#     model = CatBoostRegressor(**params, verbose=False)
    
#     # 训练模型
#     model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)
    
#     # 获取特征重要性
#     feature_importances = model.get_feature_importance()
#     important_features_indices = [i for i, importance in enumerate(feature_importances) if importance >= 0.001]
#     important_features_names = [X_train.columns[i] for i in important_features_indices]  # Assuming X_train is a DataFrame

#     # 预测
#     preds = model.predict(X_test)
#     rmse = np.sqrt(mean_squared_error(y_test, preds))
    
#     return rmse, important_features_names

# # 参数配置
# best_params = {
#     'catboost': catboost_params,
#     'xgb': xgb_params
# }

# # 评估每个大型Cluster
# for model_type, params in best_params.items():
#     print(f"Evaluating using {model_type} parameters:")
#     for cluster in [2]:
#         rmse, important_features = train_and_feature_importance(cluster, train_df, test_df, params)
#         print(f"Cluster {cluster} RMSE: {rmse}")
#         print(f"Cluster {cluster} Important features: {important_features}")


In [ ]:
Cluster 0 C/X ratio
Best ratio: 0.5 with RMSE:0.9819137040333653

Cluster 2
same params as C0
Best ratio: 0.70 with RMSE: 0.872879930616473

Cluster 3
same params as C0
Best ratio: 0.8 with RMSE: 0.9518091202077188

Cluster 4
Best ratio: 0.70 with RMSE: 0.9450906040132107

In [59]:
# from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import LinearRegression

# cluster = 2
# X_train, y_train = prepare_test_data(train_df, cluster) 
# X_test, y_test = prepare_test_data(test_df, cluster)

# from sklearn.linear_model import Ridge
# from sklearn.ensemble import StackingRegressor

# # 定义基模型
# estimators = [
#     ('xgboost', XGBRegressor(**xgb_params[cluster])),
#     ('catboost', CatBoostRegressor(**catboost_params[cluster], verbose=False))
# ]

# # 使用岭回归作为元学习器
# final_estimator = Ridge(alpha=1)

# # 创建Stacking集成模型
# stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=final_estimator, cv=3)

# # 训练模型
# stacking_regressor.fit(X_train, y_train)

# # 预测和评估
# stacking_preds = stacking_regressor.predict(X_test)
# stacking_rmse = np.sqrt(mean_squared_error(y_test, stacking_preds))
# print(f"Stacking RMSE for cluster 3: {stacking_rmse}")

Stacking RMSE for cluster 3: 0.8713790528792243


In [9]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 准备数据
X_train, y_train = prepare_test_data(train_df, 4) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

X_test, y_test = prepare_test_data(test_df, 4)

# 初始化模型使用最佳参数
cb_model = CatBoostRegressor(
    iterations=int(1486),  # 使用最佳迭代次数，转换为整数
    depth=int(13),       # 使用最佳深度，转换为整数
    learning_rate=0.06229424029098117,   # 使用最佳学习率
    l2_leaf_reg=18.150895075720104,      # 使用最佳L2正则化
    verbose=False
)

xgb_model = XGBRegressor(
    max_depth=int(6),                    # 使用最佳深度
    learning_rate=0.04246778091879101,   # 使用最佳学习率
    n_estimators=int(491),               # 使用最佳树的数量，转换为整数
    subsample=0.8689265095175513,        # 使用最佳子采样率
    objective='reg:squarederror',        # 指定目标函数
    verbosity=0
)

# 训练模型
cb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)
xgb_model.fit(X_train, y_train)

# 使用测试集进行评估
cb_preds = cb_model.predict(X_test).clip(1, 5)  # 预测值限制在1到5之间
xgb_preds = xgb_model.predict(X_test).clip(1, 5)

# 集成预测，这里使用平均集成
final_preds = 0.5 * cb_preds + 0.5 * xgb_preds

# 计算RMSE
rmse = np.sqrt(mean_squared_error(y_test, final_preds))
print(f"Test RMSE for Cluster 0: {rmse}")

Test RMSE for Cluster 0: 0.9875197193474762


In [34]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 假设 cb_preds 和 xgb_preds 已经准备好
best_rmse = float('inf')
best_ratio = 0

# 尝试不同的权重比例
for ratio in np.arange(0, 1.05, 0.05):  # 从0到1，步长为0.05
    final_preds = ratio * cb_preds + (1 - ratio) * xgb_preds
    rmse = np.sqrt(mean_squared_error(y_test, final_preds))
    if rmse < best_rmse:
        best_rmse = rmse
        best_ratio = ratio

print(f"Best ratio: {best_ratio} with RMSE: {best_rmse}")


Best ratio: 0.7000000000000001 with RMSE: 0.9450906040132107


In [ ]:
Cluster 0 C/X ratio
Best ratio: 0.5 with RMSE:0.9819137040333653

Cluster 2
same params as C0
Best ratio: 0.70 with RMSE: 0.872879930616473

Cluster 3
same params as C0
Best ratio: 0.8 with RMSE: 0.9518091202077188

Cluster 4
Best ratio: 0.70 with RMSE: 0.9450906040132107

In [ ]:
# '''using bayes to try on all data'''
# from bayes_opt import BayesianOptimization
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.feature_selection import RFECV
# from sklearn.model_selection import StratifiedKFold, KFold
# import numpy as np

# X_train, y_train = prepare_test_data(train_df, -1) 
# X_test, y_test = prepare_test_data(test_df, -1)

# def catboost_eval(depth, learning_rate, n_estimators, l2_leaf_reg):
#     # 确保参数是适当的数据类型
#     depth = int(depth)
#     n_estimators = int(n_estimators)
    
#     # 初始化和训练模型
#     model = CatBoostRegressor(
#         depth=depth,
#         learning_rate=learning_rate,
#         n_estimators=n_estimators,
#         l2_leaf_reg=l2_leaf_reg,
#         verbose=False
#     )
    
#     # 特征选择
#     selector = RFECV(model, step=1, cv=KFold(2), scoring='neg_mean_squared_error') # StratifiedKFold
#     selector.fit(X_train, y_train)
#     X_train_selected = selector.transform(X_train)
#     X_test_selected = selector.transform(X_test)

#     model.fit(X_train_selected, y_train, eval_set=(X_test_selected, y_test), early_stopping_rounds=10)
    
#     # 计算RMSE
#     predictions = model.predict(X_test_selected)
#     rmse = np.sqrt(mean_squared_error(y_test, predictions))
    
#     # 负RMSE，因为我们要最大化目标函数
#     print("Selected features indices:", selector.get_support(indices=True))
#     return -rmse

# # 参数边界
# pbounds = {
#     'depth': (3, 8),
#     'learning_rate': (0.01, 0.1),
#     'n_estimators': (100, 1000),
#     'l2_leaf_reg': (1, 10),
# }

# optimizer = BayesianOptimization(f=catboost_eval, pbounds=pbounds, random_state=42)
# optimizer.maximize(init_points=2, n_iter=5)

# print("Best parameters:", optimizer.max['params'])

### Moderate Sample Clusters (5, 7, 8) Boosting

Best RMSEs for each cluster: {5: 1.0007843912278394, 7: 1.0156949948241945, 8: 0.9990973587899347}
***best_params = {5: {'depth': 2.536426180826484, 'l2_leaf_reg': 2.1469803776077248, 'learning_rate': 0.7397939925293111}, 
               7: {'depth': 6.87137925586567, 'l2_leaf_reg': 18.323103915785264, 'learning_rate': 0.8392475333120256}, 
               8: {'depth': 2.3176722810449486, 'l2_leaf_reg': 7.572561905566506, 'learning_rate': 0.3547615081241711}}

In [5]:
best_params = {
    5: {'depth': 3, 'l2_leaf_reg': 0.415375550656062, 'learning_rate': 0.9017553809036529}, 
    7: {'depth': 6, 'l2_leaf_reg': 18.3792029910461, 'learning_rate': 0.9372003134293689}, 
    8: {'depth': 1, 'l2_leaf_reg': 7.440376345058953, 'learning_rate': 0.4049807340854126},
    6: {'depth': 2.0, 'l2_leaf_reg': 1.1835517768726047, 'learning_rate': 0.38770112704134657}
}

In [6]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

def train_and_feature_importance(cluster, train_df, test_df, importance_threshold=0.01):
    # 准备数据
    X_train, y_train = prepare_test_data(test_df, cluster)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    X_test, y_test = prepare_test_data(test_df, cluster)
    
    # 初始化模型
    model = CatBoostRegressor(**best_params[cluster])
    
    # 训练模型
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=False)
    
    # 获取特征重要性
    feature_importances = model.get_feature_importance()
    important_features_indices = [i for i, importance in enumerate(feature_importances) if importance >= importance_threshold]
    important_features_names = [X_train.columns[i] for i in important_features_indices]  # Assuming X_train is a DataFrame

    # preict
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    return rmse, important_features_names

# 评估每个Cluster并打印重要特征
for cluster in [5, 7, 8]:
    rmse, important_features = train_and_feature_importance(cluster, train_df, test_df)
    print(f"Cluster {cluster} RMSE: {rmse}")
    print(f"Cluster {cluster} Important features: {important_features}")

Cluster 5 RMSE: 1.0282390060477125
Cluster 5 Important features: ['cool', 'elite_years', 'compliment_writer', 'avg_ufc_count_per_review', 'bus_stars', 'city_encoded']
Cluster 7 RMSE: 0.6417139710941048
Cluster 7 Important features: ['score', 'average_stars', 'user_review_count', 'fans', 'useful', 'funny', 'cool', 'num_interactions', 'yelping_since', 'friends_count', 'elite_years', 'compliments', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_writer', 'compliment_photos', 'total_ufc_count_per_year', 'avg_ufc_count_per_review', 'bus_stars', 'bus_review_count', 'is_open', 'city_encoded', 'log_affinity_score']
Cluster 8 RMSE: 1.009791395972113
Cluster 8 Important features: ['average_stars', 'compliments', 'bus_stars']


In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

def prepare_data(test_df, cluster):
    if cluster == -1:
        test_cluster_data = test_df
    else:
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id', 'log_affinity_score''_affi])
    y_test = test_cluster_data['stars']
    return X_test, y_test

def train_and_feature_importance(cluster, train_df, test_df, importance_threshold=0.001):
    # 准备数据
    X_train, y_train = prepare_data(test_df, cluster)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    X_test, y_test = prepare_data(test_df, cluster)
    
    # 初始化模型
    model = CatBoostRegressor(**best_params[cluster], verbose=False)
    
    # 训练模型
    # model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=False)
    model.fit(X_train, y_train)
    # 获取特征重要性
    feature_importances = model.get_feature_importance()
    important_features_indices = [i for i, importance in enumerate(feature_importances) if importance >= importance_threshold]
    important_features_names = [X_train.columns[i] for i in important_features_indices]  # Assuming X_train is a DataFrame

    # preict
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    return rmse, important_features_names

# 评估每个Cluster并打印重要特征
for cluster in [6]:
    rmse, important_features = train_and_feature_importance(cluster, train_df, test_df)
    print(f"Cluster {cluster} RMSE: {rmse}")
    # print(f"Cluster {cluster} Important features: {important_features}")

In [16]:
train_df.columns

Index(['user_id', 'business_id', 'stars', 'score', 'average_stars', 'user_review_count', 'fans',
       'useful', 'funny', 'cool', 'num_interactions', 'yelping_since', 'friends_count',
       'elite_years', 'compliments', 'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_writer', 'compliment_photos', 'total_ufc_count_per_year',
       'avg_ufc_count_per_review', 'bus_stars', 'bus_review_count', 'is_open', 'city_encoded',
       'log_affinity_score', 'binary_affinity_score', 'Cluster'],
      dtype='object')

In [14]:
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split, KFold, GroupKFold
# from sklearn.feature_selection import RFECV

# import numpy as np
# import pandas as pd

# def train_and_feature_importance_with_rfecv(cluster, train_df, test_df, best_params):
#     # 准备数据
#     X_train, y_train = prepare_test_data(train_df, cluster)
#     X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)
#     X_test, y_test = prepare_test_data(test_df, cluster)
    
#     # 确保 X_train 是 DataFrame
#     if isinstance(X_train, np.ndarray):
#         X_train = pd.DataFrame(X_train)
#         X_val = pd.DataFrame(X_val)
#         X_test = pd.DataFrame(X_test)

#     # 初始化模型
#     model = CatBoostRegressor(**best_params[cluster], verbose=0)

#     # 使用RFECV进行特征选择
#     selector = RFECV(estimator=model, step=1, cv=KFold(5), scoring='neg_mean_squared_error', min_features_to_select=10)
#     selector.fit(X_train, y_train)
    
#     # 选择的最佳特征
#     X_train_selected = X_train.iloc[:, selector.support_]
#     X_val_selected = X_val.iloc[:, selector.support_]
#     X_test_selected = X_test.iloc[:, selector.support_]
#     important_features_names = X_train.columns[selector.support_].tolist()

#     # 在选择的特征上重新训练模型
#     # model.fit(X_train_selected, y_train, eval_set=(X_val_selected, y_val), early_stopping_rounds=10, verbose=False)
#     model.fit(X_train_selected, y_train)

#     # 预测和计算RMSE
#     preds = model.predict(X_test_selected)
#     rmse = np.sqrt(mean_squared_error(y_test, preds))

#     return rmse, important_features_names

# for cluster in [5, 7, 8]:
#     rmse, important_features = train_and_feature_importance_with_rfecv(cluster, train_df, test_df, best_params)
#     print(f"Cluster {cluster} RMSE: {rmse}")
#     print(f"Cluster {cluster} Important features: {important_features}")

In [10]:
# !pip install bayesian-optimization
!pip install imbalanced-learn

In [5]:
# 初始化一个字典来存储每个Cluster的最优模型和对应的RMSE
best_params = {}
best_rmses = {}

In [ ]:
# using XGB, 1: 1.014988119592079, 5: 1.0896717324335012, 6: 0.9023332514439544, 7: 1.0593137926993073, 8: 1.0335643721328076}

In [19]:
"""trying catboost"""
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from imblearn.over_sampling import SMOTE


for cluster in [7, 6, 8, 5]:
    train_cluster_data = train_df[train_df['Cluster'] == cluster]
    X= train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y = train_cluster_data['stars']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42)

    def prepare_test_data(test_df, cluster):
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
        y_test = test_cluster_data['stars']
        return X_test, y_test

    X_test, y_test = prepare_test_data(test_df, cluster)

    # 初始化CatBoostRegressor
    model = CatBoostRegressor(iterations=1000,
                              depth=3,
                              learning_rate=0.1,
                              loss_function='RMSE',
                              verbose=False)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

    # 验证模型效果
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f"Cluster {cluster}: RMSE = {rmse}")

Cluster 7: RMSE = 1.0367451814338553
Cluster 6: RMSE = 0.886017087790363
Cluster 8: RMSE = 1.008438304190436
Cluster 5: RMSE = 1.033039277638755


In [74]:
from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 初始化存储最佳参数和RMSE的字典
# best_params = {}
best_rmse = {}

# 定义CatBoost模型的目标函数
def catboost_eval(depth, learning_rate, l2_leaf_reg):
    params = {
        'iterations': 1000,
        'depth': int(depth),
        'learning_rate': learning_rate,
        'l2_leaf_reg': l2_leaf_reg,
        'loss_function': 'RMSE',
        'verbose': False
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=20)
    # model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return -rmse

# 进行Bayesian优化
for cluster in [5, 7, 8]:
    train_cluster_data = train_df[train_df['Cluster'] == cluster]
    X_train = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_train = train_cluster_data['stars']
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

    X_test, y_test = prepare_test_data(test_df, cluster)

    pbounds = {
        'depth': (int(best_params[cluster]['depth']), int(best_params[cluster]['depth'])),
        'learning_rate': (best_params[cluster]['learning_rate'] - 0.05, best_params[cluster]['learning_rate'] + 0.05),
        'l2_leaf_reg': (best_params[cluster]['l2_leaf_reg'] - 0.1, best_params[cluster]['l2_leaf_reg'] + 0.5)
    }

    optimizer = BayesianOptimization(f=catboost_eval, pbounds=pbounds, random_state=1)
    optimizer.maximize(init_points=300, n_iter=100)  # 初始点和迭代次数

    # 保存最佳参数和RMSE
    best_params[cluster] = optimizer.max['params']
    best_rmse[cluster] = -optimizer.max['target']

    print(f"Best parameters for Cluster {cluster}: {best_params[cluster]}")
    print(f"Best RMSE for Cluster {cluster}: {best_rmse[cluster]}")

# 输出所有最佳参数和RMSE
print("Best Parameters for each cluster:", best_params)
print("Best RMSEs for each cluster:", best_rmse)


|   iter    |  target   |   depth   | l2_lea... | learni... |
-------------------------------------------------------------
| 1         | -1.028    | 2.0       | 0.7476    | 0.8518    |
| 2         | -1.017    | 2.0       | 0.4034    | 0.861     |
| 3         | -1.017    | 2.0       | 0.5227    | 0.8914    |
| 4         | -1.023    | 2.0       | 0.5669    | 0.9203    |
| 5         | -1.025    | 2.0       | 0.8422    | 0.8545    |
| 6         | -1.022    | 2.0       | 0.5658    | 0.9076    |
| 7         | -1.022    | 2.0       | 0.4342    | 0.9318    |
| 8         | -1.023    | 2.0       | 0.5034    | 0.921     |
| 9         | -1.025    | 2.0       | 0.8521    | 0.8603    |
| 10        | -1.023    | 2.0       | 0.4173    | 0.9396    |
| 11        | -1.023    | 2.0       | 0.568     | 0.9475    |
| 12        | -1.017    | 2.0       | 0.7305    | 0.8833    |
| 13        | -1.025    | 2.0       | 0.8162    | 0.8536    |
| 14        | -1.023    | 2.0       | 0.9087    | 0.9266    |
| 15    

KeyboardInterrupt: 

In [ ]:
Best RMSEs for each cluster: {5: 1.016683932307932, 7: 1.0235422827493459, 8: 1.0067979494296004}
Best Parameters for each cluster: {5: {'depth': 2.8372035559550515, 'l2_leaf_reg': 0.415375550656062, 'learning_rate': 0.9017553809036529}, 
                                   7: {'depth': 5.629383322650899, 'l2_leaf_reg': 18.092897673552347, 'learning_rate': 0.9302409756669214}, 
                                   8: {'depth': 1.5335491744742042, 'l2_leaf_reg': 7.9441523547045145, 'learning_rate': 0.24635888070147063}}


# Best RMSEs for each cluster: {5: 1.0167557553607258, 7: 1.024030158568109, 8: 1.006788237522231}
# Best Parameters for each cluster: {5: {'depth': 2.1531204375035022, 'l2_leaf_reg': 0.9149735257647046, 'learning_rate': 0.854079466000547}, 
#                                    7: {'depth': 5.214188323944921, 'l2_leaf_reg': 18.292789406047262, 'learning_rate': 0.9229233425688939}, 
#                                    8: {'depth': 1.829594199526172, 'l2_leaf_reg': 7.93945174660862, 'learning_rate': 0.24655331821866988}}

Best RMSEs for each cluster: {5: 1.0169264030768188, 7: 1.0235736207689943, 8: 1.0052618567763993}
Best Parameters for each cluster: {5: {'depth': 2.575413872096655, 'l2_leaf_reg': 1.2907324679934653, 'learning_rate': 0.8549361230165192}, 
                                   7: {'depth': 5.595277607917133, 'l2_leaf_reg': 18.3792029910461, 'learning_rate': 0.9372003134293689}, 
                                   8: {'depth': 1.259435014043129, 'l2_leaf_reg': 7.440376345058953, 'learning_rate': 0.4049807340854126}}

# Best RMSEs for each cluster: {5: 1.0183972296867843, 7: 1.02524269156402, 8: 1.0052728101359314}
# Best Parameters for each cluster: {5: {'depth': 2.7171259457036654, 'l2_leaf_reg': 1.6469803776077248, 'learning_rate': 0.7897939925293112}, 
#                                    7: {'depth': 5.069375108517913, 'l2_leaf_reg': 17.99920470248267, 'learning_rate': 0.8756635310988201}, 
#                                    8: {'depth': 1.084407832239098, 'l2_leaf_reg': 7.730202335144113, 'learning_rate': 0.40387851258988705}}

In [9]:
Best RMSEs for each cluster: {5: 1.0007843912278394, 7: 1.0156949948241945, 8: 0.9990973587899347}
***best_params = {5: {'depth': 2.536426180826484, 'l2_leaf_reg': 2.1469803776077248, 'learning_rate': 0.7397939925293111}, 
               7: {'depth': 6.87137925586567, 'l2_leaf_reg': 18.323103915785264, 'learning_rate': 0.8392475333120256}, 
               8: {'depth': 2.3176722810449486, 'l2_leaf_reg': 7.572561905566506, 'learning_rate': 0.3547615081241711}}

In [75]:
from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 初始化存储最佳参数和RMSE的字典
# best_params = {}
best_rmse = {}

# 定义CatBoost模型的目标函数
def catboost_eval(depth, learning_rate, l2_leaf_reg):
    params = {
        'iterations': 1000,
        'depth': int(depth),
        'learning_rate': learning_rate,
        'l2_leaf_reg': l2_leaf_reg,
        'loss_function': 'RMSE',
        'verbose': False
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=20)
    # model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return -rmse

# 进行Bayesian优化
for cluster in [6]:
    train_cluster_data = train_df[train_df['Cluster'] == cluster]
    X_train = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_train = train_cluster_data['stars']
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    X_test, y_test = prepare_test_data(test_df, cluster)

    pbounds = {
        'depth': (int(best_params[cluster]['depth']), int(best_params[cluster]['depth'])),
        'learning_rate': (best_params[cluster]['learning_rate'] - 0.05, best_params[cluster]['learning_rate'] + 0.05),
        'l2_leaf_reg': (best_params[cluster]['l2_leaf_reg'] - 0.1, best_params[cluster]['l2_leaf_reg'] + 0.5)
    }

    optimizer = BayesianOptimization(f=catboost_eval, pbounds=pbounds, random_state=1)
    optimizer.maximize(init_points=200, n_iter=100)  # 初始点和迭代次数

    # 保存最佳参数和RMSE
    best_params[cluster] = optimizer.max['params']
    best_rmse[cluster] = -optimizer.max['target']

    print(f"Best parameters for Cluster {cluster}: {best_params[cluster]}")
    print(f"Best RMSE for Cluster {cluster}: {best_rmse[cluster]}")

# 输出所有最佳参数和RMSE
print("Best Parameters for each cluster:", best_params)
print("Best RMSEs for each cluster:", best_rmse)


|   iter    |  target   |   depth   | l2_lea... | learni... |
-------------------------------------------------------------
| 1         | -0.8912   | 2.0       | 1.516     | 0.3377    |
| 2         | -0.891    | 2.0       | 1.172     | 0.3469    |
| 3         | -0.89     | 2.0       | 1.291     | 0.3774    |
| 4         | -0.8947   | 2.0       | 1.335     | 0.4062    |
| 5         | -0.8885   | 2.0       | 1.61      | 0.3404    |
| 6         | -0.8887   | 2.0       | 1.334     | 0.3936    |
| 7         | -0.8917   | 2.0       | 1.202     | 0.4178    |
| 8         | -0.8934   | 2.0       | 1.272     | 0.4069    |
| 9         | -0.8916   | 2.0       | 1.62      | 0.3462    |
| 10        | -0.8915   | 2.0       | 1.185     | 0.4255    |
| 11        | -0.8925   | 2.0       | 1.336     | 0.4335    |
| 12        | -0.8884   | 2.0       | 1.499     | 0.3693    |
| 13        | -0.8884   | 2.0       | 1.584     | 0.3395    |
| 14        | -0.8885   | 2.0       | 1.677     | 0.4125    |
| 15    

In [ ]:
Best Parameters for each cluster: {6: {'depth': 2.0, 'l2_leaf_reg': 1.1835517768726047, 'learning_rate': 0.38770112704134657}}
Best RMSEs for each cluster: {6: 0.8837446917922023}

Best Parameters for each cluster: {6: {'depth': 2.61655602741076, 'l2_leaf_reg': 0.7257814607240285, 'learning_rate': 0.4030828148420926}}
Best RMSEs for each cluster: {6: 0.8842334031650307}

Best Parameters for each cluster: {6: {'depth': 2.950053661171063, 'l2_leaf_reg': 0.6486081840841198, 'learning_rate': 0.39867067648355686}}
Best RMSEs for each cluster: {6: 0.8865506932349891}

In [71]:
best_params = {6: {'depth': 2.950053661171063, 'l2_leaf_reg': 0.6486081840841198, 'learning_rate': 0.39867067648355686}}

In [ ]:
# Best Parameters for each cluster: {5: {'depth': 2.993493090480159, 'l2_leaf_reg': 1.9453159754453004, 'learning_rate': 0.736287494768294}, 
#                                    7: {'depth': 6.87137925586567, 'l2_leaf_reg': 18.323103915785264, 'learning_rate': 0.8392475333120256}, 
#                                    8: {'depth': 2.208545783377387, 'l2_leaf_reg': 8.456919746460654, 'learning_rate': 0.2832516216429279}}
# Best RMSEs for each cluster: {5: 1.0010820144199166, 7: 1.015653168191138, 8: 1.000305549514303}

# Best Parameters for each cluster: {***5: {'depth': 2.536426180826484, 'l2_leaf_reg': 2.1469803776077248, 'learning_rate': 0.7397939925293111}, 
#                                    7: {'depth': 6.716705166497869, 'l2_leaf_reg': 17.60463486698329, 'learning_rate': 0.7809900704215599}, 
#                                    ***8: {'depth': 2.2848383925693954, 'l2_leaf_reg': 7.64236558394111, 'learning_rate': 0.3545012597743415}}
# Best RMSEs for each cluster: {5: 1.0009132591349195, 7: 1.0188319686406953, 8: 0.9991016563528642}

# Best Parameters for each cluster: {5: {'depth': 2.260149256878929, 'l2_leaf_reg': 1.771921285491023, 'learning_rate': 0.7469798363027962}, 
#                                    ***7: {'depth': 6.336524926677278, 'l2_leaf_reg': 17.81766138426488, 'learning_rate': 0.8294239282966256},
#                                    8: {'depth': 3.3769352821951455, 'l2_leaf_reg': 7.652216726613778, 'learning_rate': 0.26090859187150245}}
# Best RMSEs for each cluster: {5: 1.009668889015085, 7: 1.016489858960287, 8: 1.0000324480217584}


# {5: 1.0165864148856156, 7: 1.018870661936267, 8: 1.0012668953921562}
# {5: {'depth': 3.8631427835104324, 'l2_leaf_reg': 1.080469117124645, 'learning_rate': 0.8920618912033068}, 
#  7: {'depth': 6.459387391131091, 'l2_leaf_reg': 17.52155119655796, 'learning_rate': 0.7785244285710464}, 
#  8: {'depth': 3.219613498858582, 'l2_leaf_reg': 6.9542411087094465, 'learning_rate': 0.39832986606754484}}


# Best parameters for Cluster 8: {'depth': 3.1197656703616117, 'l2_leaf_reg': 8.096715894710567, 'learning_rate': 0.2589612531162498}
# Best RMSE for Cluster 8: 1.0001124660202096

# Best parameters for Cluster 7: {'depth': 6.579716021009375, 'l2_leaf_reg': 7.396807489755532, 'learning_rate': 0.28058104702652625}
# Best RMSE for Cluster 7: 1.0270522194225222

# Best parameters for Cluster 5: {'depth': 3.503385081458421, 'l2_leaf_reg': 2.404965197063553, 'learning_rate': 0.20647001835054793}
# Best RMSE for Cluster 5: 1.0216433481698786

In [12]:
# from sklearn.ensemble import VotingRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor

# estimators = [
#     ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
#     ('lr', LinearRegression()),
#     ('dt', DecisionTreeRegressor())
# ]

# voting = VotingRegressor(estimators=estimators)
# voting.fit(X_train, y_train)
# predictions = voting.predict(X_val)
# rmse = np.sqrt(mean_squared_error(y_val, predictions))


# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(degree=2)
# X_train_poly = poly.fit_transform(X_train)
# X_val_poly = poly.transform(X_val)

# model.fit(X_train_poly, y_train)
# predictions = model.predict(X_val_poly)
# rmse = np.sqrt(mean_squared_error(y_val, predictions))
# print(rmse)

1.0846155607336996


In [ ]:
from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 遍历指定的Clusters
for cluster in [7, 6, 8, 1, 5]:
    print(f"Optimizing for Cluster {cluster}")
    # 准备数据
    train_cluster_data = train_df[train_df['Cluster'] == cluster]
    X_train = train_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_train = train_cluster_data['stars']
    
    test_cluster_data = test_df[test_df['Cluster'] == cluster]
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_test = test_cluster_data['stars']
    
    # 定义目标函数
    def cb_cv(depth, learning_rate, n_estimators, l2_leaf_reg):
        depth = int(depth)
        n_estimators = int(n_estimators)
        estimator = CatBoostRegressor(
            depth=depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            l2_leaf_reg=l2_leaf_reg,
            random_seed=42,
            verbose=False
        )
        estimator.fit(X_train, y_train)
        preds = estimator.predict(X_test)
        return -np.sqrt(mean_squared_error(y_test, preds))

    # 设置参数搜索空间
    pbounds = {
        'depth': (4, 10),
        'learning_rate': (0.005, 0.25),
        'n_estimators': (100, 1000),
        'l2_leaf_reg': (5, 15)
    }

    # pbounds = {
    #     'depth': (int(best_params[cluster]['depth']) - 1, int(best_params[cluster]['depth']) + 1),
    #     'learning_rate': (max(0.01, best_params[cluster]['learning_rate'] - 0.005), best_params[cluster]['learning_rate'] + 0.005),
    #     'n_estimators': (max(100, int(best_params[cluster]['n_estimators']) - 50), int(best_params[cluster]['n_estimators']) + 50),
    #     'l2_leaf_reg': (max(1, best_params[cluster]['l2_leaf_reg'] - 1), best_params[cluster]['l2_leaf_reg'] + 1)
    # }


    # 使用贝叶斯优化
    optimizer = BayesianOptimization(f=cb_cv, pbounds=pbounds, random_state=42, verbose=2)
    optimizer.maximize(init_points=15, n_iter=30)  # 增加初始化点和迭代次数

    # 保存最佳参数和对应的RMSE
    best_params[cluster] = optimizer.max['params']
    best_rmses[cluster] = -optimizer.max['target']

    print(f"Best parameters for Cluster {cluster}: {best_params[cluster]}")
    print(f"Best RMSE for Cluster {cluster}: {best_rmses[cluster]}")

# 输出所有最佳参数和RMSE
print("Best Parameters for each cluster:", best_params)
print("Best RMSEs for each cluster:", best_rmses)


Optimizing for Cluster 7
|   iter    |  target   |   depth   | l2_lea... | learni... | n_esti... |
-------------------------------------------------------------------------
| 1         | -1.078    | 6.247     | 14.51     | 0.1843    | 638.8     |
| 2         | -1.037    | 4.936     | 6.56      | 0.01923   | 879.6     |
| 3         | -1.039    | 7.607     | 12.08     | 0.01004   | 972.9     |
| 4         | -1.038    | 8.995     | 7.123     | 0.04955   | 265.1     |
| 5         | -1.05     | 5.825     | 10.25     | 0.1108    | 362.1     |
| 6         | -1.054    | 7.671     | 6.395     | 0.07658   | 429.7     |
| 7         | -1.042    | 6.736     | 12.85     | 0.05392   | 562.8     |
| 8         | -1.061    | 7.554     | 5.465     | 0.1538    | 253.5     |
| 9         | -1.08     | 4.39      | 14.49     | 0.2416    | 827.6     |
| 10        | -1.073    | 5.828     | 5.977     | 0.1726    | 496.1     |
| 11        | -1.036    | 4.732     | 9.952     | 0.01343   | 918.4     |
| 12        |

In [ ]:
RMSE Scores by Cluster: 1: 0.9486380392367105, 5: 1.0693467025903196, 6: 0.9071186256407098, 7: 1.042741917361027, 8: 1.039941487282683}

In [ ]:
Best Parameters for each cluster: {7: {'depth': 5.233959253839706, 'l2_leaf_reg': 5.531793633070324, 'learning_rate': 0.022523592873744964, 'n_estimators': 895.3299321377824}, 
                                   6: {'depth': 8.56956279182458, 'l2_leaf_reg': 7.727827495157177, 'learning_rate': 0.018229238234732596, 'n_estimators': 462.46324152466224}, 
                                   8: {'depth': 5.994933768167258, 'l2_leaf_reg': 5.186746727672216, 'learning_rate': 0.02510956175893532, 'n_estimators': 362.65732003713}, 
                                   1: {'depth': 6.031006169303008, 'l2_leaf_reg': 9.729298920378694, 'learning_rate': 0.012723862553242364, 'n_estimators': 162.51483915965784}, 
                                   5: {'depth': 6.031006169303008, 'l2_leaf_reg': 9.729298920378694, 'learning_rate': 0.012723862553242364, 'n_estimators': 162.51483915965784}}
Best RMSEs for each cluster: {7: 1.0374946293262677, 6: 0.8875830102874258, 8: 1.01610383204095, 1: 0.9179302653164916, 5: 1.049184245822053}

#### WORST: 8 re-model

> Best parameters for Cluster 8: {'depth': 6.352673798460998, 'l2_leaf_reg': 7.564480551391564, 'learning_rate': 0.1775612544990892, 'n_estimators': 638.3900345249591}
Best RMSE for Cluster 8: 0.3849426727018452


In [62]:
from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# 准备Cluster 8的数据
data_cluster8 = final_df[final_df['Cluster'] == 8]
X = data_cluster8.drop(columns=['stars', 'user_id', 'business_id'])
y = data_cluster8['stars']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 目标函数定义
def cb_cv(n_estimators, depth, learning_rate, l2_leaf_reg):
    depth = int(depth)
    n_estimators = int(n_estimators)
    model = CatBoostRegressor(n_estimators=n_estimators, depth=depth,
                              learning_rate=learning_rate, l2_leaf_reg=l2_leaf_reg,
                              random_seed=42, verbose=False)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    return -rmse

# 参数搜索空间设置
params = {
    'n_estimators': (600, 900),
    'depth': (6, 15),
    'learning_rate': (0.17, 0.30),
    'l2_leaf_reg': (6,10)
}

# 进行Bayesian优化
optimizer = BayesianOptimization(f=cb_cv, pbounds=params, random_state=42)
optimizer.maximize(init_points=5, n_iter=100)

# 输出最优参数
print("Best parameters:", optimizer.max['params'])
best_params = optimizer.max['params']
best_model = CatBoostRegressor(n_estimators=int(best_params['n_estimators']),
                               depth=int(best_params['depth']),
                               learning_rate=best_params['learning_rate'],
                               l2_leaf_reg=best_params['l2_leaf_reg'],
                               random_seed=42, verbose=False)
best_model.fit(X_train, y_train)
best_preds = best_model.predict(X_val)
final_rmse = np.sqrt(mean_squared_error(y_val, best_preds))
print("Final RMSE for Cluster 8:", final_rmse)


|   iter    |  target   |   depth   | l2_lea... | learni... | n_esti... |
-------------------------------------------------------------------------
| 1         | -1.08     | 9.371     | 9.803     | 0.2652    | 779.6     |
| 2         | -1.076    | 7.404     | 6.624     | 0.1776    | 859.9     |
| 3         | -1.06     | 11.41     | 8.832     | 0.1727    | 891.0     |
| 4         | -1.071    | 13.49     | 6.849     | 0.1936    | 655.0     |
| 5         | -1.066    | 8.738     | 8.099     | 0.2262    | 687.4     |
| 6         | -1.069    | 12.36     | 8.929     | 0.209     | 890.4     |
| 7         | -1.063    | 10.64     | 9.268     | 0.2448    | 891.0     |
| 8         | -1.055    | 11.07     | 8.206     | 0.17      | 891.6     |
| 9         | -1.051    | 10.44     | 7.285     | 0.19      | 890.9     |
| 10        | -1.083    | 11.13     | 6.36      | 0.2904    | 892.1     |
| 11        | -1.059    | 10.64     | 7.172     | 0.2105    | 890.3     |
| 12        | -1.056    | 10.03     | 

In [60]:
# from catboost import CatBoostRegressor
# from sklearn.model_selection import cross_val_score
# best_param_set = best_params[1]
# best_param_set['n_estimators'] = round(best_param_set['n_estimators'])
# best_param_set['depth'] = round(best_param_set['depth'])

# model = CatBoostRegressor(**best_param_set)

# # 使用交叉验证来评估模型
# scores = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
# print("Cross-validated RMSE scores for Cluster 1:", -scores)
# print("Average RMSE:", -scores.mean())

In [59]:
# import matplotlib.pyplot as plt

# # 对预测结果进行可视化
# plt.figure(figsize=(10, 6))
# plt.scatter(y_val, y_pred, alpha=0.3)
# plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=4)
# plt.xlabel('Measured')
# plt.ylabel('Predicted')
# plt.title('Validation Data Predicted vs Actual Values')
# plt.show()


### Small Sample Cluster (1)

In [174]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

def prepare_data(df, cluster, drop_columns):
    if cluster == -1:
        cluster_data = df
    else:
        cluster_data = df[df['Cluster'] == cluster]

    # 确保尝试删除的列存在于DataFrame中，否则忽略
    drop_columns = [col for col in drop_columns if col in cluster_data.columns]
    X = cluster_data.drop(columns=drop_columns, errors='ignore')
    y = cluster_data['stars']
    return X, y

def train_simple_model(cluster, train_df, test_df):
    drop_columns = ['stars', 'user_id', 'business_id']
    # 如果score列有NaN值，添加到删除列表
    if pd.isnull(train_df['score']).any() or pd.isnull(test_df['score']).any():
        drop_columns.append('score')
        
    X_train, y_train = prepare_data(train_df, cluster, drop_columns)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.50, random_state=42)
    X_test, y_test = prepare_data(test_df, cluster, drop_columns)
    
    # 使用岭回归并通过交叉验证选择最佳正则化系数
    model = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=KFold(5))
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    return rmse

# 计算Cluster 1和6的RMSE
rmse_1 = train_simple_model(1, train_df, val_df)
print(f"Cluster 1 RMSE: {rmse_1}")

Cluster 1 RMSE: 1.3309021302671094


In [175]:
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.impute import SimpleImputer

def prepare_data(df, cluster, drop_columns):
    if cluster == -1:
        cluster_data = df
    else:
        cluster_data = df[df['Cluster'] == cluster]

    # 确保尝试删除的列存在于DataFrame中，否则忽略
    drop_columns = [col for col in drop_columns if col in cluster_data.columns]
    X = cluster_data.drop(columns=drop_columns, errors='ignore')
    y = cluster_data['stars']
    return X, y

def train_elastic_net(cluster, train_df, test_df):
    drop_columns = ['stars', 'user_id', 'business_id']
    # 如果score列有NaN值，添加到删除列表
    if pd.isnull(train_df['score']).any() or pd.isnull(test_df['score']).any():
        drop_columns.append('score')

    X_train, y_train = prepare_data(train_df, cluster, drop_columns)
    X_test, y_test = prepare_data(test_df, cluster, drop_columns)
        
    # 使用ElasticNetCV自动调整参数
    model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=np.logspace(-6, 2, 100), cv=5, random_state=42)
    model.fit(X_train, y_train)
    
    print("Best alpha:", model.alpha_)
    print("Best l1_ratio:", model.l1_ratio_)
    
    # 训练最终模型
    model = ElasticNet(alpha=model.alpha_, l1_ratio=model.l1_ratio_)
    model.fit(X_train, y_train)
    
    # 预测测试集
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    return rmse

cluster = 1  # 示例使用Cluster 1
rmse = train_elastic_net(cluster, train_df, val_df)
print(f"Cluster {cluster} RMSE: {rmse}")

Best alpha: 0.0035938136638046293
Best l1_ratio: 0.1
Cluster 1 RMSE: 1.3274108290306523


In [181]:
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_selection import SelectFromModel

def prepare_data(df, cluster, drop_columns):
    if cluster == -1:
        cluster_data = df
    else:
        cluster_data = df[df['Cluster'] == cluster]

    drop_columns = [col for col in drop_columns if col in cluster_data.columns]
    X = cluster_data.drop(columns=drop_columns, errors='ignore')
    y = cluster_data['stars']
    return X, y

def train_elastic_net(cluster, train_df, test_df):
    drop_columns = ['stars', 'user_id', 'business_id']
    if train_df['score'].isnull().any() or test_df['score'].isnull().any():
        drop_columns.append('score')

    X_train, y_train = prepare_data(train_df, cluster, drop_columns)
    X_test, y_test = prepare_data(test_df, cluster, drop_columns)
        
    # 使用ElasticNetCV进行更细粒度的参数搜索
    model_cv = ElasticNetCV(l1_ratio=np.linspace(0.01, 0, 1), alphas=np.logspace(-6, 1, 50), cv=5, random_state=42)
    model_cv.fit(X_train, y_train)
    
    print("Best alpha:", model_cv.alpha_)
    print("Best l1_ratio:", model_cv.l1_ratio_)
    
    # 使用选择的最佳参数训练ElasticNet模型
    model = ElasticNet(alpha=model_cv.alpha_, l1_ratio=model_cv.l1_ratio_, random_state=42)
    model.fit(X_train, y_train)
    
    # 特征选择
    selector = SelectFromModel(estimator=model, threshold='mean', prefit=True)
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    # 重新训练模型使用被选择的特征
    model.fit(X_train_selected, y_train)
    
    # 预测并计算RMSE
    preds = model.predict(X_test_selected)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    return rmse

# 示例代码执行
cluster = 1  # 使用Cluster 1
rmse = train_elastic_net(cluster, train_df, test_df)
print(f"Cluster {cluster} RMSE: {rmse}")

Best alpha: 0.005179474679231208
Best l1_ratio: 0.01
Cluster 1 RMSE: 0.9858956922327305


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [165]:
import pandas as pd

def check_nan_columns(df):
    nan_columns = df.columns[df.isna().any()].tolist()
    if nan_columns:
        print("以下列包含NaN值：")
        for column in nan_columns:
            print(column)
    else:
        print("DataFrame中没有包含NaN值的列。")

# 检查val_df中的NaN列
check_nan_columns(val_df)

以下列包含NaN值：
score


In [127]:
def train_elastic_net(cluster, train_df, test_df):
    # 准备数据
    X_train, y_train = prepare_data(train_df, cluster)
    X_test, y_test = prepare_data(test_df, cluster)
    
    # 使用ElasticNetCV自动调整参数
    model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=np.logspace(-6, 2, 100), cv=5, random_state=42)
    model.fit(X_train, y_train)
    
    print("Best alpha:", model.alpha_)
    print("Best l1_ratio:", model.l1_ratio_)
    
    # 训练最终模型
    final_model = ElasticNet(alpha=model.alpha_, l1_ratio=model.l1_ratio_)
    final_model.fit(X_train, y_train)
    
    # 预测测试集
    test_preds = final_model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
    
    return test_rmse

cluster = 1  # 示例使用Cluster 1
rmse = train_elastic_net(cluster, train_df, test_df)
print(f"Cluster {cluster} RMSE: {rmse}")


Best alpha: 0.0035938136638046293
Best l1_ratio: 0.1
Cluster 1 RMSE: 0.9591603171535215


# SPECIFIC FEATURES AND CLUSTER ANALYSIS

# RANK METHOD

In [51]:
# 计算每个Cluster中有多少条记录
group_data = final_df.groupby('Cluster').size().to_list()

In [58]:
# 准备XGBoost的数据，假设label是目标变量
X = final_df.drop(columns=['stars', 'user_id', 'business_id'])

y = final_df['stars']

In [59]:
# final_df.columns

In [60]:
# 创建DMatrix
dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group_data)

params = {
    'objective': 'rank:pairwise',
    'eta': 0.1,
    'gamma': 0.1,
    'min_child_weight': 0.1,
    'max_depth': 3
}

# 训练模型
bst = xgb.train(params, dtrain, num_boost_round=10)

In [62]:
## Prepare Test data 
test_data_file = '../yelp_true.csv'
test_df = pd.read_csv(test_data_file)

data_folder_path = '../data/'
test_res_rdd = feature_processor.process_all_features(sc, test_df, data_folder_path, test_data_file)

val_df = rdd_to_pandas(test_res_rdd)
val_df = val_df.merge(user_clusters, on='user_id', how='left')

val_group_data = val_df.groupby('Cluster').size().to_list()

X_val = val_df.drop(columns=['stars', 'user_id', 'business_id'])

y_val = val_df['stars']

In [63]:
dtest = xgb.DMatrix(X_val, label=y_val)

In [64]:
predictions = bst.predict(dtest)
print("Predictions:", predictions)

Predictions: [-0.5655462  -0.39554703 -0.379111   ... -0.28824937 -0.37578714
 -0.63751644]


## Evaluate

In [75]:
from sklearn.model_selection import train_test_split

X = merged_final_df.drop(['user_id', 'business_id', 'stars'], axis=1)
y = merged_final_df['stars']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
params = {
    'objective': 'reg:squarederror',
    'colsample_bytree': 0.6000253271077364,
    'gamma': 0.3476073435753331,
    'learning_rate': 0.03186356519406673,
    'max_depth': 8,
    'n_estimators': 480,
    'subsample': 0.8831766281686719
}

best_model = XGBRegressor(**params)

best_model.fit(X_train, y_train)

# 使用独立的测试集进行最终评估
y_pred = best_model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"{test_data_file}的RMSE:", final_rmse)

../yelp_true.csv的RMSE: 0.9819683713608058
